Train a CNN model for beamforming using hybrid supervised and unsupervised training

In [108]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from typing import List, Dict
import importlib
# Get the current working directory
scripts_dir = os.getcwd()
# Go up one level
project_root = os.path.abspath(os.path.join(scripts_dir, '..'))
sys.path.append(project_root)

import src.CNN
importlib.reload(src.CNN)
from src.CNN import ChannelCNN, Trainer

In [109]:
class setup():
    def __init__(self, n_tx, n_rx:int, num_streams:int, num_users, PT):
        self.n_tx = n_tx
        self.n_rx = n_rx
        self.d = num_streams
        self.K = num_users
        self.PT = PT

In [110]:
# Define the setup
set_up = setup(n_tx=8, n_rx=2, num_streams=2, num_users=4, PT=5)

In [111]:
# Defien the CNN model and the trainer
cn = ChannelCNN(set_up)
tr = Trainer(set_up, cn)

In [112]:
# Define the dataset dataframe
def complex_tensor():
    real = torch.randn(2, 8)
    imag = torch.randn(2, 8)
    return real + 1j * imag

# Create the DataFrame
H_df = pd.DataFrame([[complex_tensor() for _ in range(4)] for _ in range(10)])

def complex_tensor():
    real = torch.randn(8, 2)
    imag = torch.randn(8, 2)
    return real + 1j * imag

# Create the DataFrame
V_df = pd.DataFrame([[complex_tensor() for _ in range(4)] for _ in range(10)])

dataset = pd.concat([H_df, V_df], axis=1)

In [113]:
# Supervised training
tr.train_supervised(dataset=dataset, num_epochs=10, batch_size=2)

Epoch 1/10, Loss: 0.8949359655380249
Epoch 2/10, Loss: 0.8633920550346375
Epoch 3/10, Loss: 0.8344792723655701
Epoch 4/10, Loss: 0.7984554767608643
Epoch 5/10, Loss: 0.739816427230835
Epoch 6/10, Loss: 0.6784834861755371
Epoch 7/10, Loss: 0.6114283800125122
Epoch 8/10, Loss: 0.537505030632019
Epoch 9/10, Loss: 0.4539209008216858
Epoch 10/10, Loss: 0.36555784940719604


In [114]:
# Unsupervised training
tr.train_unsupervised(dataset=dataset, num_epochs=10, batch_size=2)

/Users/Ali/Projects/Deep-Unfolding-NN/src/CNN.py:124: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  H = H.applymap(lambda x: x.to(torch.cfloat))


Epoch 1/10, Loss: 1405.818115234375
Epoch 2/10, Loss: 156.85256958007812
Epoch 3/10, Loss: 4.6633219718933105
Epoch 4/10, Loss: -3.9695911407470703
Epoch 5/10, Loss: -3.989509344100952
Epoch 6/10, Loss: -4.021352767944336
Epoch 7/10, Loss: -4.060084819793701
Epoch 8/10, Loss: -4.104735851287842
Epoch 9/10, Loss: -4.157665252685547
Epoch 10/10, Loss: -4.221340179443359
Training complete!
